# ¡Query Time!

Ahora, vamos a comparar operaciones parecidas con ambas bases de datos.

Comencemos haciendo un pequeño set-up:

- El **firewall**! Este notebook tiene una dirección IP diferente, así que debemos agregarla a las reglas de Firewall **¿Alguien me puede guiar?**
- **Instalar** las **librerías** necesarias
- **Conectarse** a las bases de datos

In [28]:
!curl ipecho.net/plain

34.60.162.179

In [29]:
!pip install psycopg2-binary pymongo # incluimos en el firewall la ip del notebook para conectarlo con mongod, tuvimos que hacer cambios en la consola de Mongo

### Vamos a conectarnos a nuestra DB Postgre:

Necesitamos la IP!
Vayamos a SQL -> Instances a ver nuestra DB

In [30]:
dbname='postgres'
ip='34.88.249.70' # Tu IP

print("Conectando a la siguiente base de datos POSTGRE:")
print("dbname={} user=dota password=password host={}".format(dbname, ip))

Conectando a la siguiente base de datos POSTGRE:
dbname=postgres user=dota password=password host=34.88.249.70


In [31]:
import psycopg2 #conectamos el notebook con psgrade SQL (nos acabamos de conectar antes con mongo tambien )
conn = psycopg2.connect("dbname={} user=dota password=password host={}".format(dbname, ip))
postgres = conn.cursor()

Espera... ¿Qué pasa? 🤔

Estamos utilizando otro notebook! Tiene otra IP!

- Editemos la configuración de nuestra instancia de Postgre: Entremos en la configuración y en Connections -> Add a Network

In [32]:
import psycopg2 #No esta conectada el notebook a mongo
conn = psycopg2.connect()
postgres = conn.cursor()

OperationalError: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?


### Conectémonos a nuestra DB Mongo:

En ocasiones, si tenemos algún error con la conexión al server de Mongo, los problemas pueden tener varios orígenes:
- Nuestra IP ha cambiado
- La IP de la máquina virtual ha cambiado
- Algún proceso de Mongo se ha quedado colgado en el servidor y debemos reiniciarlo (´"sudo systemctl restart mongod" en la consola SSH de la máquina)

In [26]:
from pymongo import MongoClient
from pprint import pprint
client = MongoClient('34.147.84.54', 27017, directConnection=True)
mongodb = client.dota

# Querys sencillas:

Ahora que ya estamos conectados a ambas Bases de Datos, comencemos con algunas consultas sencillas:



### Postgre:

In [16]:
# Vamos a hacer un select en la tabla heroes para que nos devuelva la información de Tiny
postgres = conn.cursor()
postgres.execute("SELECT * FROM heroes WHERE name = 'Tiny'")
postgres.fetchone()

(122,
 'Tiny',
 'Strength',
 "Target Radius: 275 Damage: 25/35/45/55  Movement Speed Increase: 20/40/60  Dota Allstars: Looming from the rocky crags of Ashenra, the stone giants are born from the mountain itself. Tiny, a newly born giant, although small in size, has the strength of a hill. With his supreme strength, he is able to launch anything into the air, including huge amounts of soil to cause avalanches around his enemies. Because of his craggy exterior, enemies have a hard time attacking him. Over time, Tiny's body will grow bigger as the magnetic forces deep inside his granite heart pull rocks and soil to his rugged stone form. Dota 2: Coming to life as a chunk of stone, Tiny's origins are a mystery on which he continually speculates. He is a Stone Giant now, but what did he used to be? A splinter broken from a Golem's heel? A shard swept from a gargoyle-sculptor's workshop? A fragment of the Oracular Visage of Garthos? A deep curiosity drives him, and he travels the world tire

### Mongo:

In [27]:
mongodb.heroes.find_one({'name': 'Tiny'})

ServerSelectionTimeoutError: 34.147.84.54:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 67c9fb00c71aaf49716b1748, topology_type: Single, servers: [<ServerDescription ('34.147.84.54', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('34.147.84.54:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

# Querys con **LIKE**

### PostgreSQL

In [19]:
# Hagamos una consulta que nos devuelva todo aquello de la tabla items donde la descripción contiene blade y más cosas
postgres.execute("SELECT * FROM items WHERE description ilike '%blade%'")
postgres.fetchone()

(4,
 'Blades_of_Attack',
 'The damage of these small, concealable blades should not be underestimated.',
 ' +9 Damage',
 450,
 datetime.datetime(2017, 6, 13, 20, 23, 51, 730051))

### MongoDB

In [ ]:
mongodb.heroes.find_one({"lore": {"$regex": ".*blade.*"}}) #Lore es un documento dentro de los documentos de mongo

# Count

# PostgreSQL

In [34]:
# Vamos a devolver el número de filas que existen en la tabla heroes cuando tipo sea Strength
postgres.execute("SELECT COUNT(1) FROM heroes where type = 'Strength'")# CONTINÚA LA CONSULTA...
postgres.fetchone()

(25,)

### MongoDB

In [35]:
mongodb.heroes.count_documents({"type":"Strength"} #CONTINÚA LA CONSULTA})

SyntaxError: incomplete input (<ipython-input-35-12c055552670>, line 1)

# Nuevas Colecciones/Tablas

### PostgreSQL

In [36]:
# Crea la tabla quests con cuatro columnas: ID, name, description, y exp.

try: #indicar dentro de un paquete try para poder evitar problemas que bloqueen
  postgres.execute("""
CREATE TABLE quests(
   ID           INT PRIMARY KEY NOT NULL,
   name         CHAR(50) NOT NULL,
   description  CHAR(250) NOT NULL,
   exp          INT NOT NULL
)""")
  conn.commit() # si esta bien hace un commit
except: # sino sale bien reintenta ejecutarlo
  conn.rollback()
  print("table exists")

postgres.execute("SELECT * FROM quests")
postgres.fetchone()

### MongoDB

In [ ]:
# Inserta un nuevo documento en la colección quests y luego recupera el primer documento de esa colección.

mongodb.quests.insert_one({ #mencionamos una nueva coleccion no existente de nombre quests y asi la crea
    "name": "find sword",
    "description": "Find the hidden sword",
    "exp": 500
    })
mongodb.quests.find_one()

In [ ]:
mongodb.quests.insert_one({ #no hace falta crear la coleccion , mete el documento en quest(que ya existe)
    "result": 500,
    "id": 1
    })

mongodb.quests.find_one({"id": 1}) #busca el id que sea 1

# Inserción de datos

### PostgreSQL

In [37]:
# Vamos a insertar: INSERT INTO quests (id, name, description, exp) VALUES (100, 'find sword', 'Find the hidden sword', 500)
try:
  postgres.execute("""
  INSERT INTO quests (id, name, description, exp) VALUES (100, 'find sword', 'Find the hidden sword', 500)
  """)
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")


postgres.execute("select * from quests")
postgres.fetchone()

(100,
 'find sword                                        ',
 'Find the hidden sword                                                                                                                                                                                                                                     ',
 500)

### MongoDB

In [ ]:
# Ahora insertemos en Mongo al personaje con nombre: Squall y valor: 1 en realidad se han incorporado 2 docs
mongodb.quests.insert_many([
    {
        "nombre": "Squall"
    },
    {
        "valor": 1
     }
])
quests = mongodb.quests.find()
for quest in quests:
  print(quest)

__NOTA:__ Los documentos en MongoDB no tienen restricciones de insercion por primary key (se duplican).

# Joins

### PostgreSQL

In [38]:
# Vamos a buscar en todas las columnas de las tablas heroes y skills cuando el nombre sea 'Pugna'
#evitar bloqueos con el try except

try:
  postgres.execute("""
  SELECT * FROM heroes
  LEFT JOIN skills on skills.hero_id = heroes.id
  WHERE heroes.name = 'Pugna'
  """)
  heroes_skills = postgres.fetchall()
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")

for hero in heroes_skills:
    pprint(hero)

(139,
 'Pugna',
 'Intelligence',
 'More information required... Dota 2:',
 datetime.datetime(2017, 6, 13, 20, 20, 18, 898552),
 523,
 'Nether Blast',
 'An exploding pulse deals damage to enemies and structures in the area. Deals '
 'half damage to structures. The Arts of Oblivion include a deafening blast of '
 'emerald flames from the Nether Reaches.\n',
 139,
 datetime.datetime(2017, 6, 13, 20, 20, 18, 919084))
(139,
 'Pugna',
 'Intelligence',
 'More information required... Dota 2:',
 datetime.datetime(2017, 6, 13, 20, 20, 18, 898552),
 524,
 'Decrepify',
 'A powerful banishing spell that slows a unit and renders it unable to attack '
 'or be attacked. Afflicted units take extra magic damage. A now-mastered '
 'relic from his childhood, Pugna delights in banishing others into the Nether '
 'Realm, whether for good, evil, or simple enjoyment.\n',
 139,
 datetime.datetime(2017, 6, 13, 20, 20, 18, 919811))
(139,
 'Pugna',
 'Intelligence',
 'More information required... Dota 2:',
 dateti

### MongoDB

In [ ]:
mongodb.heroes.find_one( # facilidad de busqueda en mongo , no hace falta joins .Se podria hacer tambien try except, aunque el riesgo es menor por se una consulta
    {
        "name": "Pugna"
    }
    )

**No hay que hacer Joins!**

# Modificar tablas - Añadir columnas

### PostgreSQL

In [41]:
# Vamos a agegar a la tabla heroes la columna 'favorite_items'

try:
  postgres.execute("""
  alter table heroes add column favorite_items varchar(300) default
  """      )
  conn.commit()
except:
  conn.rollback()
  print("ALTER ALREADY DONE BY OTHER STUDENT")

ALTER ALREADY DONE BY OTHER STUDENT


In [ ]:
# Vamos a actualizar el valor de la tabla heroes estableciendo el valor de la columna 'favorite_items'
# a 'Teleport Scroll, magic wand, sword' cuando el name sea 'Tiny'
try:
  postgres.execute(#Aquí tu código#hay que incluir aqui el codigo
                   )
  conn.commit()
except:
  conn.rollback()
  print("UPDATE ALREADY DONE BY OTHER STUDENT")

postgres.execute("select * from heroes where name = 'Tiny'")
postgres.fetchone()

### MongoDB

In [ ]:
# actualiza todos los documentos en la colección heroes donde el campo "name" es "Tiny" poniendo como "favorite_items" 'Teleport Scroll, magic wand, sword'

mongodb.heroes.update_many( #funciona con documentos Json
	#Aquí tu código
	 {"name": "Tiny"},
	 {"$Set": {"favorite_items": "Teleport Scroll, magic wand, sword" }}
)
mongodb.heroes.find_one({"name": "Tiny"})

# Relacionar datos nuevos entre tablas/colecciones

### PostgreSQL

In [42]:
# Vamos a ponerle al heroe con id 33 el item con id 26 -> insertar una nueva fila en la tabla heroes_items con los valores hero_id = 33 e item_id = 26
try:
  postgres.execute(
      """Insert into heroes_items (heroe_id, item_id) Values(33,26)
      """             )
  conn.commit()
except:
  conn.rollback()
  print("INSERT ALREADY DONE BY OTHER STUDENT")

postgres.execute("select * from heroes_items")
postgres.fetchone()

INSERT ALREADY DONE BY OTHER STUDENT


In [43]:
postgres.execute("""
    SELECT * FROM heroes
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 33;
""")
postgres.fetchone()

(33,
 'Goblin_Techies',
 'Intelligence',
 '',
 datetime.datetime(2017, 6, 12, 16, 11, 30, 469745),
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

### MongoDB

In [ ]:
# Aquí la relación se hace de otra forma! -> Ahora vamos a agregar al heroe con name 'Visage' el item 'Blades_of_Attack'

mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$push" """metes una nueva relacion por eso no es set """ : {"items": mongodb.items.find_one({"name": "Blades_of_Attack"})}}
)
mongodb.heroes.find_one({"name": "Visage"})

# Modificar datos existentes

### PostgreSQL

In [ ]:
# Vamos a actualizar el valor de Sacred_Relic a MEGA SWORD
try:
  postgres.execute(#Aquí tu código#
                   """ update items set name = " MEGA SWORD" WHERE
                   )
  conn.commit()
except:
  conn.rollback()
  print("UPDATE ALREADY DONE BY OTHER STUDENT")

In [ ]:
postgres.execute("""
    SELECT * FROM heroes
    LEFT JOIN heroes_items ON heroes.id = heroes_items.hero_id
    LEFT JOIN items ON heroes_items.item_id = items.id
    WHERE heroes.id = 33
""")
postgres.fetchone()

### MongoDB

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$pull": {"items": {"name": "Blades_of_Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$set": {"name": "MEGA Blades of Attack"}}
)
mongodb.heroes.find_one({"name": "MEGA Blades of Attack"})

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$push": {"items": mongodb.items.find_one({"name": "Visage"})}}
)
mongodb.heroes.find_one(
    {
    "name": "MEGA Blades of Attack"
    })

# Borrar elementos

### PostgreSQL

In [ ]:
# Borremos el elemento MEGA SWORD de la tabla items
try:
  postgres.execute(#Aquí tu código#
                   )
  conn.commit()
except:
  conn.rollback()
  print("DELETE ALREADY DONE BY OTHER STUDENT")

NOTA: Al fallar, lo ideal seria cambiar el constraint:


```
ALTER TABLE table_y   
    DROP CONSTRAINT constraint_name,   
    ADD CONSTRAINT constraint_name FOREIGN KEY (column_in_table_y)
          REFERENCES table_x (referenced_column_in_table_x) ON DELETE CASCADE;
```

### MongoDB

In [ ]:
mongodb.items.delete_one( {"name": "MEGA Blades of Attack"} )
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

In [ ]:
mongodb.heroes.update_one(
	{"name": "Visage"},
	{"$pull": {"items": {"name": "MEGA Blades of Attack"}}}
)
mongodb.heroes.find_one(
    {
    "name": "Visage"
    })

# Agrupar (GROUP BY / ORDER BY)

### PostgreSQL

In [ ]:
# Cuenta cuántos héroes hay por cada tipo en la tabla heroes y los ordena de manera descendente.

postgres.execute("""
SELECT type, count(*) FROM heroes GROUP BY type ORDER BY type DESC;
""")
hero_types = postgres.fetchall()

for hero in hero_types:
    pprint(hero)

### MongoDB

In [ ]:
# Agregación para contar cuántos héroes hay por cada tipo en la colección heroes

aggregation = mongodb.heroes.aggregate([
    {"$group" : {"_id": "$type", "count": {"$sum": 1}}}
])
for results in aggregation:
    print(results)